# Libraries Import

In [1]:
from sklearn.model_selection import KFold
from IPython.display import display
import random
import numpy as np
import pandas as pd

# Lectura de datos

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
df = pd.read_csv('./data.csv')

In [5]:
df

,ID,Title,Keywords,FirstLevel
0,1,Integration of Communities into Process-Orient...,"cooperative knowledge generation,knowledge co...",H
1,3,Small Groups Learning Synchronously Online at ...,"professional training,workplace learning,compu...",H
2,3,Small Groups Learning Synchronously Online at ...,"professional training,workplace learning,compu...",J
3,4,Using Weblogs for Knowledge Sharing and Learni...,"Experience-based Information System,wiki,weblo...",A
4,4,Using Weblogs for Knowledge Sharing and Learni...,"Experience-based Information System,wiki,weblo...",D
...,...,...,...,...
1899,1476,Visualization of Syntax Trees for Language Pro...,"syntax trees,visualization,educational software",D
1900,1476,Visualization of Syntax Trees for Language Pro...,"syntax trees,visualization,educational software",K
1901,1477,"Learning to Program with COALA, a Distributed ...","learning programming,intelligent tutoring system",D
1902,1477,"Learning to Program with COALA, a Distributed ...","learning programming,intelligent tutoring system",I


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1904 entries, 0 to 1903
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          1904 non-null   int64 
 1   Title       1904 non-null   object
 2   Keywords    1863 non-null   object
 3   FirstLevel  1904 non-null   object
dtypes: int64(1), object(3)
memory usage: 59.6+ KB


In [7]:
isHadNull = df.isnull().sum()

print(isHadNull)

ID             0
Title          0
Keywords      41
FirstLevel     0
dtype: int64


### Eliminando valores nulos

In [8]:
df_clean = df.dropna().reset_index(drop=True)

In [9]:
df_clean

,ID,Title,Keywords,FirstLevel
0,1,Integration of Communities into Process-Orient...,"cooperative knowledge generation,knowledge co...",H
1,3,Small Groups Learning Synchronously Online at ...,"professional training,workplace learning,compu...",H
2,3,Small Groups Learning Synchronously Online at ...,"professional training,workplace learning,compu...",J
3,4,Using Weblogs for Knowledge Sharing and Learni...,"Experience-based Information System,wiki,weblo...",A
4,4,Using Weblogs for Knowledge Sharing and Learni...,"Experience-based Information System,wiki,weblo...",D
...,...,...,...,...
1858,1476,Visualization of Syntax Trees for Language Pro...,"syntax trees,visualization,educational software",D
1859,1476,Visualization of Syntax Trees for Language Pro...,"syntax trees,visualization,educational software",K
1860,1477,"Learning to Program with COALA, a Distributed ...","learning programming,intelligent tutoring system",D
1861,1477,"Learning to Program with COALA, a Distributed ...","learning programming,intelligent tutoring system",I


### Agrupando las Categorias por Articulo

In [11]:
grouped_df = df_clean.groupby(['ID', 'Title', 'Keywords'])['FirstLevel'].unique().reset_index()
grouped_df = grouped_df.sort_values('ID')
grouped_df

,ID,Title,Keywords,FirstLevel
0,1,Integration of Communities into Process-Orient...,"cooperative knowledge generation,knowledge co...",[H]
1,3,Small Groups Learning Synchronously Online at ...,"professional training,workplace learning,compu...","[H, J]"
2,4,Using Weblogs for Knowledge Sharing and Learni...,"Experience-based Information System,wiki,weblo...","[A, D, H, J, K]"
3,5,Modelling and Implementing Pre-built Informati...,"modelling method,introduction method,context-a...","[H, I, J]"
4,6,Tube Map Visualization: Evaluation of a Novel ...,"knowledge visualization,information visualiza...",[H]
...,...,...,...,...
1111,1473,Eduquito: Virtual Environment for Digital Incl...,"virtual environment,accessibility,persons with...",[K]
1112,1474,Development of a Web Application for Managemen...,"learning styles,Web-based application",[K]
1113,1475,Semantic Spiral Timelines Used as Support for ...,"visualization,e-learning,timeline,spiral,Moodle","[L, M]"
1114,1476,Visualization of Syntax Trees for Language Pro...,"syntax trees,visualization,educational software","[D, K]"


### Verificando la cantidad de datos por categoria

In [15]:
df_categories = pd.DataFrame({'Categories': df['FirstLevel'].drop_duplicates(), 'Values': {}}).sort_values('Categories').reset_index(drop=True)
g = df_clean['FirstLevel'].value_counts()

i__ = 0
for key, value in g.items():
    mask = df_categories['Categories'] == key
    df_categories.loc[mask, 'Values'] = value
    i__+=1

df_categories

,Categories,Values
0,A,35.0
1,B,50.0
2,C,123.0
3,D,313.0
4,E,55.0
5,F,301.0
6,G,110.0
7,H,379.0
8,I,229.0
9,J,87.0


# Preprocesamiento de Datos 🔍

## Import librerias nltk (Natural Language Toolkit)

In [16]:
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kpzao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kpzao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kpzao\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kpzao\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Estableciendo separador para cada token

In [17]:
def textProcessingToString(row):
  strText = ""
  for i in range(len(row)):
    # print(text)
    if (i == len(row) -1):
      strText = strText + row[i]
    else:
      strText = strText + row[i] + "##"
  return strText

### Stemming and Stop Word

In [18]:
def textProcessingWithStopwordsAndStemming(df_data):

  titleData = df_data['Title'].to_numpy()
  keywordsData = df_data['Keywords'].to_numpy()

  n = len(titleData)

  titlePre = []
  keywordsPre = []

  for i in range(n):
    titleText = titleData[i].strip()
    keywordText = keywordsData[i].strip()

    titleTokens = word_tokenize(titleText)
    keywordTokens = word_tokenize(keywordText)
    keywordTokens = [item for item in keywordTokens if item != ',']

    #CLEAR DATA
    titleTokens = [string for string in titleTokens if len(string) > 1]
    keywordTokens = [string for string in keywordTokens if len(string) > 1]

    # Filtrar los tokens que solo contengan letras y números
    titleTokens = [token for token in titleTokens if re.match(r'^[a-zA-Z]+$', token)]
    keywordTokens = [token for token in keywordTokens if re.match(r'^[a-zA-Z]+$', token)]

    #STOPWORDS
    # Obtener la lista de stop words en inglés
    stop_words = set(stopwords.words('english'))

    # Filtrar el texto removiendo las stop words
    filteredtTitle = [w for w in titleTokens if not w.lower() in stop_words]
    filteredtKeyword = [w for w in keywordTokens if not w.lower() in stop_words]

    #STEMMING
    stemmer = PorterStemmer()

    # Aplicar stemming a cada palabra del texto
    stemmedTitle = [stemmer.stem(word) for word in filteredtTitle]
    stemmedKeyword = [stemmer.stem(word) for word in filteredtKeyword]

    titlePre.append(stemmedTitle)
    keywordsPre.append(stemmedKeyword)

  return titlePre, keywordsPre

In [19]:
%%time
titlePre,keywordsPre = textProcessingWithStopwordsAndStemming(grouped_df)

CPU times: total: 1.12 s
Wall time: 1.13 s


### Procesando la Data

In [20]:
def setTitleKeywordClean():

  titlePre,keywordsPre = textProcessingWithStopwordsAndStemming(grouped_df)

  #by Title
  newData = []
  for row in titlePre:
    titleKey = textProcessingToString(row)
    # Eliminar los puntos utilizando regex
    titleText_cleaned = re.sub(r'\.', '', titleKey)
    newData.append(titleText_cleaned)

  #by Keywords
  newData = []
  for row in keywordsPre:
    titleKey = textProcessingToString(row)
    # Eliminar los puntos utilizando regex
    titleText_cleaned = re.sub(r'\.', '', titleKey)
    newData.append(titleText_cleaned)

  grouped_df['Keywords_clean'] = newData
  grouped_df['Title_clean'] = newData

In [21]:
setTitleKeywordClean()

In [22]:
grouped_df

,ID,Title,Keywords,FirstLevel,Keywords_clean,Title_clean
0,1,Integration of Communities into Process-Orient...,"cooperative knowledge generation,knowledge co...",[H],cooper##knowledg##gener##knowledg##commun##pro...,cooper##knowledg##gener##knowledg##commun##pro...
1,3,Small Groups Learning Synchronously Online at ...,"professional training,workplace learning,compu...","[H, J]",profession##train##workplac##learn##cooper##le...,profession##train##workplac##learn##cooper##le...
2,4,Using Weblogs for Knowledge Sharing and Learni...,"Experience-based Information System,wiki,weblo...","[A, D, H, J, K]",inform##system##wiki##weblog##pedagog##inform#...,inform##system##wiki##weblog##pedagog##inform#...
3,5,Modelling and Implementing Pre-built Informati...,"modelling method,introduction method,context-a...","[H, I, J]",model##method##introduct##method##inform##retr...,model##method##introduct##method##inform##retr...
4,6,Tube Map Visualization: Evaluation of a Novel ...,"knowledge visualization,information visualiza...",[H],knowledg##visual##inform##visual##visual##meta...,knowledg##visual##inform##visual##visual##meta...
...,...,...,...,...,...,...
1111,1473,Eduquito: Virtual Environment for Digital Incl...,"virtual environment,accessibility,persons with...",[K],virtual##environ##access##person##disabl,virtual##environ##access##person##disabl
1112,1474,Development of a Web Application for Managemen...,"learning styles,Web-based application",[K],learn##style##applic,learn##style##applic
1113,1475,Semantic Spiral Timelines Used as Support for ...,"visualization,e-learning,timeline,spiral,Moodle","[L, M]",visual##timelin##spiral##moodl,visual##timelin##spiral##moodl
1114,1476,Visualization of Syntax Trees for Language Pro...,"syntax trees,visualization,educational software","[D, K]",syntax##tree##visual##educ##softwar,syntax##tree##visual##educ##softwar


# Obtener el mejor thresold

In [23]:
def getThresold(population,df_featuresMap,nColumns,nInd,nGenes,dataFBRM_train,exp_list,threshold):
  # aca la funcion fittness sera trabajoso basicamente es el algoritmo fuzzy classifier algorithm mas la parte de evaluacion multietiqueta

  fitnessByChromosome =[]
  thresholdByChromosome =[]

  cnt_p = 1

  for chromosome in population:

    bestThresold = 0
    bestAcurracy = 0
    print("\n ------------------------------------ POBLACION: ", cnt_p," ------------------------------------ \n")
    thres = threshold
    cont = 0
    while(thres < 1.0):

      pred_list_kt = []
      pred_list_k = []
      pred_list_t = []
      print("Con threshold: ", thres);

      cnt_d = 1
      for tf in dataFBRM_train:
        # print("\nDOCUMENTO TF",cnt_d,"\n")
        pred_keywords_title, pred_keywords, pred_title = FuzzyClassifierAlgorithm(chromosome,df_featuresMap,nColumns,tf,thres)
        pred_list_kt.append(pred_keywords_title)
        pred_list_k.append(pred_keywords)
        pred_list_t.append(pred_title)
        cnt_d += 1


      labels = sorted(set([label for sublist in exp_list for label in sublist]))

      y_true_bin = np.zeros((len(exp_list), len(labels)))
      y_pred_bin = np.zeros((len(pred_list_kt), len(labels)))

      for i, label_list in enumerate(exp_list):
          for label in label_list:
              y_true_bin[i, labels.index(label)] = 1

      for i, label_list in enumerate(pred_list_kt):
          for label in label_list:
              y_pred_bin[i, labels.index(label)] = 1

      # Calcular la precisión (accuracy) para cada etiqueta y promediar
#       macro_accuracy = 0
#       for i in range(len(labels)):
#           true_positives = np.sum(np.logical_and(y_true_bin[:, i], y_pred_bin[:, i]))
#           true_negatives = np.sum(np.logical_and(1 - y_true_bin[:, i], 1 - y_pred_bin[:, i]))
#           total_samples = true_positives + true_negatives + np.sum(np.logical_xor(y_true_bin[:, i], y_pred_bin[:, i]))
#           label_accuracy = (true_positives + true_negatives) / total_samples
#           macro_accuracy += label_accuracy

#       macro_accuracy /= len(labels)
      macro_f1 = 0
      for i in range(len(labels)):
          true_positives = np.sum(np.logical_and(y_true_bin[:, i], y_pred_bin[:, i]))
          false_positives = np.sum(np.logical_and(1 - y_true_bin[:, i], y_pred_bin[:, i]))
          false_negatives = np.sum(np.logical_and(y_true_bin[:, i], 1 - y_pred_bin[:, i]))

          precision = true_positives / (true_positives + false_positives + 1e-9)
          recall = true_positives / (true_positives + false_negatives + 1e-9)

          f1_score = 2 * (precision * recall) / (precision + recall + 1e-9)
          macro_f1 += f1_score

      macro_f1 /= len(labels)

      if(macro_f1 > bestAcurracy):

        bestThresold = thres
        bestAcurracy = macro_f1
        print("the best Thresold: ", bestThresold)
        print("the best Acurracy: ", bestAcurracy)
        cont = 0
      else:
        cont +=1
      
      if (cont > 5):
            break;

      thres+=0.01

    cnt_p +=1
    fitnessByChromosome.append(bestAcurracy)
    thresholdByChromosome.append(bestThresold)

  return fitnessByChromosome, thresholdByChromosome

#  Fuzzy Based Rules Merger (FBRM) algorithm

### Obtener la Frecuencia de Terminos por Articulo (TF)

In [24]:
def count_terms_frecuency(r):

  freq_dist = FreqDist(r)
  term_frequency = dict(freq_dist)

  return term_frequency

In [25]:
def algorithmFBRM(category,data_):
  k = [] #array of keywords of category c
  t = [] #array of title of category c
  r = [] # resultant array of k and t of category c

  for i in range(len(data_)):
    titles = data_.iloc[i]['Title_clean']
    keywords = data_.iloc[i]['Keywords_clean']

    titles = re.split(r'##', titles)
    keywords = re.split(r'##', keywords)

    for text in titles:
      k.append(text)
    for text in keywords:
      t.append(text)
  r = k + t
  return k, t, r

# ALGORITMO GENETICO

### Cantidad de parametros
> Cantidad de individuos: 10

> Cantidad de genes por individuos: ---

> Seleccion por torneo

> Probabilidad de Cruzamiento: 0.9

> Cruzamiento de BLX-alpha :0.5

> Probabilidad de mutacion: 0.5

> Mutacion de Intercambio

> Cantidad de iteraciones: 2000

## Generacion de Poblacion

In [26]:
def genPopulation(nInd,data_tf):
  datadf_chromosome = data_tf[data_tf['Metadata'] != 'tf_keywords_title']
  # print(datadf_chromosome)
  df_features = datadf_chromosome.iloc[:, 2:]
  # print(df_features.shape)
  population = []
  flattened_list = df_features.values.ravel().tolist()
  # print(flattened_list)

  for i in range(nInd):
    # population.append([int(round(random.uniform(gen - (gen * 0.1), gen + (gen * 0.1)))) for gen in flattened_list])
    # population.append([random.uniform(gen - (gen * 0.1), gen + (gen * 0.1)) for gen in flattened_list])
    # -1 y 1
    population.append([gen + ((random.random() * 2 - 1) * gen * 0.1) for gen in flattened_list])

  return np.array(population), df_features.shape[1]

In [27]:
# nGenes = population.shape[1]
# nGenes

## Funcion Fitness

In [28]:
def FuzzyClassifierAlgorithm(chromosome, df_featuresMap, nColumns, tf, threshold):
    pred_keywords_title = []
    pred_keywords = []
    pred_title = []

    columns = list(df_featuresMap.iloc[:, 2:].columns)
    nRow = len(chromosome) // nColumns

    tf_keywords_title = df_featuresMap[df_featuresMap['Metadata'] == 'tf_keywords_title']

    w_keywords = np.zeros(len(tf_keywords_title))
    w_title = np.zeros(len(tf_keywords_title))
    w_keywords_title = np.zeros(len(tf_keywords_title))

    c_list = tf_keywords_title['Names'].tolist()

    tf_i = 0
    tf_j = 0

    for i in range(nRow):
        if (i % 2 == 0):
          # to keyword
            idx_arr = [[columns.index(key), value] for key,value in tf.items() if key in columns]
            wk_k = sum([chromosome[i*nColumns + idx[0]] * idx[1] for idx in idx_arr])
            w_keywords[tf_i] = wk_k
            tf_i += 1
        else:
          # to title
            idx = [columns.index(key) for key,value in tf.items() if key in columns]
            wk_t = sum([chromosome[i*nColumns + idx[0]] * idx[1] for idx in idx_arr])
            w_title[tf_j] = wk_t
            tf_j += 1

    tf_tk = 0
    # to keyword + title
    for i in range(nRow):
        if (i % 2 == 0):
            idx = [columns.index(key) for key,value in tf.items() if key in columns]
            wk_kt = sum([(chromosome[i*nColumns + idx[0]] + chromosome[(i+1)*nColumns + idx[0]]) * idx[1] for idx in idx_arr])
            w_keywords_title[tf_tk] = wk_kt
            tf_tk += 1


    # print("KEYWORDS")
    # print(w_keywords)
    # print("TITLE")
    # print(w_title)
    # print("KEYWORD + TITLE")
    # print(w_keywords_title)

    weightSum = w_keywords.sum()
    uD = w_keywords / weightSum
    pred_keywords = [c_list[i] for i in range(len(uD)) if uD[i] >= threshold]

    weightSum = w_title.sum()
    uD = w_title / weightSum
    pred_title = [c_list[i] for i in range(len(uD)) if uD[i] >= threshold]

    weightSum = w_keywords_title.sum()
    uD = w_keywords_title / weightSum
    pred_keywords_title = [c_list[i] for i in range(len(uD)) if uD[i] >= threshold]

    return pred_keywords_title, pred_keywords, pred_title


In [29]:
def getFittness(population,df_featuresMap,nColumns,nInd,nGenes,dataFBRM_train,exp_list,threshold):
  # aca la funcion fittness sera trabajoso basicamente es el algoritmo fuzzy classifier algorithm mas la parte de evaluacion multietiqueta

#   fitnessByChromosome =[]
#   predictionList = []
#   thresholdByChromosome =[]

#   cnt_p = 1
#   for chromosome in population:

#     bestThresold = 0
#     bestAcurracy = 0
#     print("\n ------------------------------------ POBLACION: ", cnt_p," ------------------------------------ \n")
#     thres = threshold
#     cont = 0
#     while(thres < 1.0):

#       pred_list_kt = []
#       pred_list_k = []
#       pred_list_t = []
#       print("Con threshold: ", thres);

#       cnt_d = 1
#       for tf in dataFBRM_train:
#         # print("\nDOCUMENTO TF",cnt_d,"\n")
#         pred_keywords_title, pred_keywords, pred_title = FuzzyClassifierAlgorithm(chromosome,df_featuresMap,nColumns,tf,thres)
#         pred_list_kt.append(pred_keywords_title)
#         pred_list_k.append(pred_keywords)
#         pred_list_t.append(pred_title)
#         cnt_d += 1


#       labels = sorted(set([label for sublist in exp_list for label in sublist]))

#       y_true_bin = np.zeros((len(exp_list), len(labels)))
#       y_pred_bin = np.zeros((len(pred_list_kt), len(labels)))

#       for i, label_list in enumerate(exp_list):
#           for label in label_list:
#               y_true_bin[i, labels.index(label)] = 1

#       for i, label_list in enumerate(pred_list_kt):
#           for label in label_list:
#               y_pred_bin[i, labels.index(label)] = 1

#       # Calcular la precisión (accuracy) para cada etiqueta y promediar
#       macro_accuracy = 0
#       for i in range(len(labels)):
#           true_positives = np.sum(np.logical_and(y_true_bin[:, i], y_pred_bin[:, i]))
#           true_negatives = np.sum(np.logical_and(1 - y_true_bin[:, i], 1 - y_pred_bin[:, i]))
#           total_samples = true_positives + true_negatives + np.sum(np.logical_xor(y_true_bin[:, i], y_pred_bin[:, i]))
#           label_accuracy = (true_positives + true_negatives) / total_samples
#           macro_accuracy += label_accuracy

#       macro_accuracy /= len(labels)
# #       macro_f1 = 0
# #       for i in range(len(labels)):
# #           true_positives = np.sum(np.logical_and(y_true_bin[:, i], y_pred_bin[:, i]))
# #           false_positives = np.sum(np.logical_and(1 - y_true_bin[:, i], y_pred_bin[:, i]))
# #           false_negatives = np.sum(np.logical_and(y_true_bin[:, i], 1 - y_pred_bin[:, i]))

# #           precision = true_positives / (true_positives + false_positives + 1e-9)
# #           recall = true_positives / (true_positives + false_negatives + 1e-9)

# #           f1_score = 2 * (precision * recall) / (precision + recall + 1e-9)
# #           macro_f1 += f1_score

# #       macro_f1 /= len(labels)

#       if(macro_accuracy > bestAcurracy):

#         bestThresold = thres
#         bestAcurracy = macro_accuracy
#         print("the best Thresold: ", bestThresold)
#         print("the best Acurracy: ", bestAcurracy)
#         cont = 0
#       else:
#         cont +=1
      
#       if (cont > 5):
#             break;

#       thres+=0.01

#     cnt_p +=1
#     fitnessByChromosome.append(bestAcurracy)
#     thresholdByChromosome.append(bestThresold)
#     predictionList.append(pred_list_kt)

#   return fitnessByChromosome, thresholdByChromosome, predictionList
    
  fitnessByChromosome = []
  predictionList = []

  cnt_p = 1


  for chromosome in population:

    print("\n ------------------------------------ POBLACION: ", cnt_p," ------------------------------------ \n")

    pred_list_kt = []
    pred_list_k = []
    pred_list_t = []

    cnt_d = 1
    for tf in dataFBRM_train:
      # print("\nDOCUMENTO TF",cnt_d,"\n")
      pred_keywords_title, pred_keywords, pred_title = FuzzyClassifierAlgorithm(chromosome,df_featuresMap,nColumns,tf,threshold)
      pred_list_kt.append(pred_keywords_title)
      pred_list_k.append(pred_keywords)
      pred_list_t.append(pred_title)
      cnt_d += 1

    labels = sorted(set([label for sublist in exp_list for label in sublist]))

    y_true_bin = np.zeros((len(exp_list), len(labels)))
    y_pred_bin = np.zeros((len(pred_list_kt), len(labels)))

    for i, label_list in enumerate(exp_list):
        for label in label_list:
            y_true_bin[i, labels.index(label)] = 1

    for i, label_list in enumerate(pred_list_kt):
        for label in label_list:
            y_pred_bin[i, labels.index(label)] = 1

    # Calcular la precisión (accuracy) para cada etiqueta y promediar
#     macro_accuracy = 0
#     for i in range(len(labels)):
#      true_positives = np.sum(np.logical_and(y_true_bin[:, i], y_pred_bin[:, i]))
#      true_negatives = np.sum(np.logical_and(1 - y_true_bin[:, i], 1 - y_pred_bin[:, i]))
#      total_samples = true_positives + true_negatives + np.sum(np.logical_xor(y_true_bin[:, i], y_pred_bin[:, i]))
#      label_accuracy = (true_positives + true_negatives) / total_samples
#      macro_accuracy += label_accuracy

#     macro_accuracy /= len(labels)
    # Calculate the precision, recall, and F1-score for each label and average them for macro average F1-score
    macro_f1 = 0
    for i in range(len(labels)):
       true_positives = np.sum(np.logical_and(y_true_bin[:, i], y_pred_bin[:, i]))
       false_positives = np.sum(np.logical_and(1 - y_true_bin[:, i], y_pred_bin[:, i]))
       false_negatives = np.sum(np.logical_and(y_true_bin[:, i], 1 - y_pred_bin[:, i]))

       precision = true_positives / (true_positives + false_positives + 1e-9)
       recall = true_positives / (true_positives + false_negatives + 1e-9)

       f1_score = 2 * (precision * recall) / (precision + recall + 1e-9)
       macro_f1 += f1_score

    macro_f1 /= len(labels)
    cnt_p +=1

    fitnessByChromosome.append(macro_f1)
    predictionList.append(pred_list_kt) 
  return fitnessByChromosome, predictionList
    


## Seleccion por Torneo

In [30]:
def __maximize__(arr,x,y):
  idx = x if (arr[x] >= arr[y]) else y
  # print(arr[x] ,"\t- ", arr[y], "\t=> ",arr[idx])
  return idx

In [31]:
# by tournament
def selection(population,arrFittness,nInd):

  print("\nCreacion de Mating Pool\n")
  mating_pool = []

  for i in range(nInd):
    idxCouple1 = random.randint(0, nInd-1)
    idxCouple2 = random.randint(0, nInd-1)
    idxMax = __maximize__(arrFittness,idxCouple1,idxCouple2)
    mating_pool.append(population[idxMax])
    print(idxCouple1 ,"\t- ", idxCouple2, "\t=> ", idxMax, "\t=> ", population[idxMax])

    # print(idxCouple1)
    # print(idxCouple2)
    # print("menor: ",)
  return np.array(mating_pool)

## CROSSOVER

In [32]:
def blend_crossover_blx_alpha(father1,father2,pCrossover_BLX_Alpha,pCrossover):
    # offspring  = np.zeros_like(father1)
    c1 = 0
    c2 = 0
    nGenes = len(father1)
    betha = random.uniform((-1) * pCrossover_BLX_Alpha,1 + pCrossover_BLX_Alpha)

    child = []
    for i in range(nGenes):
      newGen = father1[i] + (betha*abs(father2[i] - father1[i]))
      child.append(newGen)
    return np.array(child)

In [33]:
# by blx-alpha
def crossover(mating_pool,pCrossover,nInd,cross,pCrossover_BLX_Alpha):

  child = []
  parent = []
  for i in range(nInd):
    idxCouple1 = random.randint(0, nInd-1)
    idxCouple2 = random.randint(0, nInd-1)

    print("\nSELECCION DE PADRES")
    couple_1 = mating_pool[idxCouple1]
    couple_2 = mating_pool[idxCouple2]
    print(idxCouple1, " - ", idxCouple2, " =>  ", couple_1, " - ", couple_2)

    if (random.random() <= pCrossover):
      offstring = cross(couple_1,couple_2,pCrossover_BLX_Alpha,pCrossover)
      child.append(offstring)
      print("\nCRUZAMIENTO")
      print(offstring)

    else:
      print("\nSin Cruzamiento")
      parent.append(couple_1)


  return np.array(child), np.array(parent)

## MUTATION of 10% with value float

In [34]:
def mutate(children,pMutation,nGenes):

  for i in range(len(children)):
    for j in range(len(children[i])):
      if (random.random() <= pMutation):

        # print("\nMUTACION")
        gen = children[i][j]
        # print("Posicion: ", j, " => ", children[i][j], "\t=> valor actual: ", gen)
        # children[i][j] = round(random.uniform(gen - (gen * 0.1), gen + (gen * 0.1)))
        children[i][j] = gen + ((random.random() * 2 - 1) * gen * 0.1)
        # population.append([gen + (random.random() * gen * 0.1) for gen in flattened_list])
        # print("Mutated: ", j, " => ", children[i][j], "\t=> valor mutado: ", children[i][j], "\n")
      # else :
        # print("\nSin Mutacion\n")
  return children

## Crossover and Mutate

In [35]:
def crossover_mutate(mating_pool,pCrossover,nInd,pMutation,nGenes,cross,pCrossover_BLX_Alpha):
  children, parents = crossover(mating_pool,pCrossover,nInd,cross,pCrossover_BLX_Alpha)

  print("\nMUTATION\n")

  if (len(children) == 0):
    return parents
  childrenMuta = mutate(children,pMutation,nGenes)

  if(len(parents) == 0):
    return childrenMuta
  offstrings = np.concatenate((childrenMuta, parents), axis=0)

  return offstrings

## Funcion del Algoritmo Genetico

In [36]:
def geneticAlgorithms(train_data = "",nInd = 16,df_featuresMap = "",dataFBRM_train_kt = "",pCrossover = 0.7,pMutation = 0.05,iteration = 100,cross = "", pCrossover_BLX_Alpha = 0.5,threshold = 0.1):

  #Seleccion de individuos nIndiM
  newGeneration, nColumns = genPopulation(nInd,df_featuresMap)
  ## numero de genes
  nGenes = newGeneration.shape[1]
  ### lista de categorias
  exp_list = train_data['FirstLevel'].tolist()


  print("OBTENER EL MEJOR THRESHOLD")
  ### obtener el mejor threshold
  candidateGeneration = []
  idxChromosome = random.randint(0, nInd-1)
  candidateGeneration.append(newGeneration[idxChromosome])

  fitnessByChromosome, thresholdByChromosome = getThresold(candidateGeneration,df_featuresMap,nColumns,nInd,nGenes,dataFBRM_train_kt,exp_list,threshold)
  print("mejores resultados: ",fitnessByChromosome, "\t-> mejor thresold",thresholdByChromosome)
  idx_max= fitnessByChromosome.index(max(fitnessByChromosome))
  bestThresold = thresholdByChromosome[idx_max]


  for i in range(iteration):
    print("\n-------ITERACION ",i+1," ----------\n")

    #Calcular Fittness
    f_, predictionList = getFittness(newGeneration,df_featuresMap,nColumns,nInd,nGenes,dataFBRM_train_kt,exp_list,bestThresold)

    print("\n NUEVA GENERACION ")
    print("Generación \t\t-> fitness")
    print(newGeneration, "\t\t->", f_)

    # Seleccion de Mating pool
    mating_pool = selection(newGeneration,f_,nInd)

    # Seleccion de padres
    # para cruzamiento
    # mutacion
    newGeneration = crossover_mutate(mating_pool,pCrossover,nInd,pMutation,nGenes,cross,pCrossover_BLX_Alpha)

  print("\n\nCondicion de Parada")

  f_, predictionList = getFittness(newGeneration,df_featuresMap,nColumns,nInd,nGenes,dataFBRM_train_kt,exp_list,bestThresold)

  print("\n\nFIN DEL TRAINING\n\n")

  return newGeneration, f_, bestThresold, predictionList

# TRAINING DATA

In [37]:
def getTFByCategories(data):
  dataFBRM = {}
  for index, row in df_categories.iterrows():
    k, t, r = algorithmFBRM(row,data[data['FirstLevel'].apply(lambda x: row['Categories'] in x)])
    tf_keywords = count_terms_frecuency(k)
    tf_title = count_terms_frecuency(t)
    tf_keywords_title = count_terms_frecuency(r)
    dataFBRM[row['Categories']] = {'tf_keywords': tf_keywords, 'tf_title': tf_title, 'tf_keywords_title': tf_keywords_title}

  print(dataFBRM['B']['tf_keywords_title']['system']) # test
  return dataFBRM

In [38]:
def getFeaturesMap(dataFBRM):

  ### Seleccionando caracteristicas
  featuresSelected = []
  for key, value in dataFBRM.items():
    feature = list(value['tf_keywords_title'].keys())
    featuresSelected += feature

  featuresSelected = set(featuresSelected)
  featuresSelected = list(featuresSelected)
  featuresSelected.sort()

  ### nuevo dataframe
  df_featuresMap = pd.DataFrame(columns=['Names'] + ['Metadata'] + featuresSelected)
  data_list = []
  for category, metadata in dataFBRM.items():
      for key, features in metadata.items():
          features['Names'] = category
          features['Metadata'] = key
          data_list.append(features)

  df_featuresMap = pd.concat([df_featuresMap, pd.DataFrame(data_list)], ignore_index=True)

  ### Preprocesando el dataframe de caracteristicas
  df_featuresMap = df_featuresMap.fillna(0)
  df_features= df_featuresMap.iloc[:, 2:]
  df_description= df_featuresMap.iloc[:, :2]

  sumCol = df_features.sum()
  df_features = df_features[sumCol.sort_values(ascending=False).index]

  df_featuresMap = pd.concat([df_description, df_features], axis=1)
  display(df_featuresMap)
  return df_featuresMap

In [39]:
def getTFByDoc(train_data):

    dataFBRM_train_kt = []
    dataFBRM_train_k = []
    dataFBRM_train_t = []

    for i in range(len(train_data)):
      # print(row)
      train_df = pd.DataFrame([train_data.iloc[i]])
      k, t, r = algorithmFBRM(0,train_df)
      tf_keywords = count_terms_frecuency(k)
      tf_title = count_terms_frecuency(t)
      tf_keywords_title = count_terms_frecuency(r)
      dataFBRM_train_kt.append(tf_keywords_title)
      dataFBRM_train_k.append(tf_keywords)
      dataFBRM_train_t.append(tf_title)

    return dataFBRM_train_k, dataFBRM_train_t, dataFBRM_train_kt

In [41]:
nInd = 10
# nGenes = 56342
pCrossover = 0.5
pMutation = 0.9
pCrossover_BLX_Alpha = 0.5
threshold = 0.1
pMutation = 0.5
iteration = 100

In [42]:
%%time
k = 3
# kf = KFold(n_splits=k,)
kf = KFold(n_splits=k, shuffle=True, random_state=42)

train_list = []
test_list = []

oldDf_FeaturesMap = []
bestOffstring_list = []
bestThresold_list = []
bestPredictionList = []

f_list = []

for train_index, test_index in kf.split(grouped_df):
    train_data, test_data = grouped_df.iloc[train_index], grouped_df.iloc[test_index]

    ## save data
    train_list.append(train_data)
    test_list.append(test_data)

    ## TF
    dataFBRM = getTFByCategories(train_data)

    ## Create a Features Map
    df_featuresMap = getFeaturesMap(dataFBRM)
    oldDf_FeaturesMap.append(df_featuresMap)

    ### TF de cada articulo
    dataFBRM_train_k, dataFBRM_train_t, dataFBRM_train_kt = getTFByDoc(train_data)

    ## Entrenamiento con Algoritmo Genetic0
    bestOffstring, f_, bestThresold, predictionList = geneticAlgorithms(train_data,nInd,df_featuresMap,dataFBRM_train_kt,pCrossover,pMutation,iteration,blend_crossover_blx_alpha,pCrossover_BLX_Alpha,threshold)
    bestOffstring_list.append(bestOffstring)
    bestThresold_list.append(bestThresold)
    bestPredictionList.append(predictionList)
    f_list.append(f_)

2


,Names,Metadata,system,comput,knowledg,model,inform,manag,languag,program,...,gather,gazett,gender,geodes,gif,recombin,recogn,glass,reattempt,zrtp
0,A,tf_keywords,5.0,1.0,27.0,2,6.0,22.0,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,5.0,1.0,27.0,2,6.0,22.0,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,10.0,2.0,54.0,4,12.0,44.0,12.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,1.0,8.0,0.0,2,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,1.0,8.0,0.0,2,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,2.0,16.0,0.0,4,0.0,0.0,6.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,16.0,13.0,3.0,12,4.0,4.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,C,tf_title,16.0,13.0,3.0,12,4.0,4.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,C,tf_keywords_title,32.0,26.0,6.0,24,8.0,8.0,6.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
9,D,tf_keywords,36.0,19.0,14.0,31,6.0,12.0,39.0,55.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


OBTENER EL MEJOR THRESHOLD

 ------------------------------------ POBLACION:  1  ------------------------------------ 

Con threshold:  0.1
the best Thresold:  0.1
the best Acurracy:  0.4525410971683296
Con threshold:  0.11
Con threshold:  0.12
Con threshold:  0.13
Con threshold:  0.14
Con threshold:  0.15000000000000002
Con threshold:  0.16000000000000003
mejores resultados:  [0.4525410971683296] 	-> mejor thresold [0.1]

-------ITERACION  1  ----------


 ------------------------------------ POBLACION:  1  ------------------------------------ 


 ------------------------------------ POBLACION:  2  ------------------------------------ 


 ------------------------------------ POBLACION:  3  ------------------------------------ 


 ------------------------------------ POBLACION:  4  ------------------------------------ 


 ------------------------------------ POBLACION:  5  ------------------------------------ 


 ------------------------------------ POBLACION:  6  ---------------------

,Names,Metadata,system,knowledg,comput,inform,manag,model,languag,program,...,fit,flood,flowchart,focu,forens,riemannian,riemann,result,fubini,self
0,A,tf_keywords,1.0,22.0,0.0,10.0,13.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,1.0,22.0,0.0,10.0,13.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,2.0,44.0,0.0,20.0,26.0,0.0,14.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,2.0,0.0,6.0,0.0,0.0,2.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,2.0,0.0,6.0,0.0,0.0,2.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,4.0,0.0,12.0,0.0,0.0,4.0,8.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,24.0,11.0,18.0,9.0,15.0,7.0,4.0,5.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,C,tf_title,24.0,11.0,18.0,9.0,15.0,7.0,4.0,5.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,C,tf_keywords_title,48.0,22.0,36.0,18.0,30.0,14.0,8.0,10.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,D,tf_keywords,44.0,10.0,19.0,9.0,8.0,24.0,30.0,55.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


OBTENER EL MEJOR THRESHOLD

 ------------------------------------ POBLACION:  1  ------------------------------------ 

Con threshold:  0.1
the best Thresold:  0.1
the best Acurracy:  0.45648912540903913
Con threshold:  0.11
Con threshold:  0.12
the best Thresold:  0.12
the best Acurracy:  0.4576881239867377
Con threshold:  0.13
Con threshold:  0.14
Con threshold:  0.15000000000000002
Con threshold:  0.16000000000000003
Con threshold:  0.17000000000000004
Con threshold:  0.18000000000000005
mejores resultados:  [0.4576881239867377] 	-> mejor thresold [0.12]

-------ITERACION  1  ----------


 ------------------------------------ POBLACION:  1  ------------------------------------ 


 ------------------------------------ POBLACION:  2  ------------------------------------ 


 ------------------------------------ POBLACION:  3  ------------------------------------ 


 ------------------------------------ POBLACION:  4  ------------------------------------ 


 ----------------------------

,Names,Metadata,system,comput,knowledg,inform,manag,model,program,languag,...,hdr,head,hermitian,hertz,restor,heyt,hine,histor,hmm,semir
0,A,tf_keywords,6,1.0,33.0,12.0,21.0,2,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,6,1.0,33.0,12.0,21.0,2,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,12,2.0,66.0,24.0,42.0,4,2.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,1,8.0,0.0,0.0,0.0,2,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,1,8.0,0.0,0.0,0.0,2,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,2,16.0,0.0,0.0,0.0,4,4.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,20,13.0,8.0,7.0,11.0,7,4.0,5.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,C,tf_title,20,13.0,8.0,7.0,11.0,7,4.0,5.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,C,tf_keywords_title,40,26.0,16.0,14.0,22.0,14,8.0,10.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,D,tf_keywords,32,22.0,4.0,11.0,10.0,33,62.0,35.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


OBTENER EL MEJOR THRESHOLD

 ------------------------------------ POBLACION:  1  ------------------------------------ 

Con threshold:  0.1
the best Thresold:  0.1
the best Acurracy:  0.4645794512565725
Con threshold:  0.11
Con threshold:  0.12
Con threshold:  0.13
Con threshold:  0.14
Con threshold:  0.15000000000000002
Con threshold:  0.16000000000000003
mejores resultados:  [0.4645794512565725] 	-> mejor thresold [0.1]

-------ITERACION  1  ----------


 ------------------------------------ POBLACION:  1  ------------------------------------ 


 ------------------------------------ POBLACION:  2  ------------------------------------ 


 ------------------------------------ POBLACION:  3  ------------------------------------ 


 ------------------------------------ POBLACION:  4  ------------------------------------ 


 ------------------------------------ POBLACION:  5  ------------------------------------ 


 ------------------------------------ POBLACION:  6  ---------------------

In [43]:
print(len(train_list))
print(len(test_list))

3
3


In [44]:
train_list[0]

,ID,Title,Keywords,FirstLevel,Keywords_clean,Title_clean
0,1,Integration of Communities into Process-Orient...,"cooperative knowledge generation,knowledge co...",[H],cooper##knowledg##gener##knowledg##commun##pro...,cooper##knowledg##gener##knowledg##commun##pro...
1,3,Small Groups Learning Synchronously Online at ...,"professional training,workplace learning,compu...","[H, J]",profession##train##workplac##learn##cooper##le...,profession##train##workplac##learn##cooper##le...
4,6,Tube Map Visualization: Evaluation of a Novel ...,"knowledge visualization,information visualiza...",[H],knowledg##visual##inform##visual##visual##meta...,knowledg##visual##inform##visual##visual##meta...
5,7,Reconciling Knowledge Management and Workflow ...,"workflow,knowledge management",[H],workflow##knowledg##manag,workflow##knowledg##manag
7,9,"KMDL - Capturing, Analysing and Improving Know...","Process-oriented Knowledge Management,knowledg...","[D, H, I]",knowledg##manag##busi##process##knowledg##mode...,knowledg##manag##busi##process##knowledg##mode...
...,...,...,...,...,...,...
1108,1470,A Meta-modeling based Approach for the Multi-D...,"model-driven development,multi-disciplinary de...","[D, K]",develop##design##educ##design##web##design,develop##design##educ##design##web##design
1110,1472,TQ-Bot: An AIML-based Tutor and Evaluator Bot,"AIML,ALICE,BDI,chatterbots,Claroline,Intellige...","[I, K]",aiml##alic##bdi##chatterbot##clarolin##intelli...,aiml##alic##bdi##chatterbot##clarolin##intelli...
1112,1474,Development of a Web Application for Managemen...,"learning styles,Web-based application",[K],learn##style##applic,learn##style##applic
1113,1475,Semantic Spiral Timelines Used as Support for ...,"visualization,e-learning,timeline,spiral,Moodle","[L, M]",visual##timelin##spiral##moodl,visual##timelin##spiral##moodl


In [45]:
test_list[0]

,ID,Title,Keywords,FirstLevel,Keywords_clean,Title_clean
2,4,Using Weblogs for Knowledge Sharing and Learni...,"Experience-based Information System,wiki,weblo...","[A, D, H, J, K]",inform##system##wiki##weblog##pedagog##inform#...,inform##system##wiki##weblog##pedagog##inform#...
3,5,Modelling and Implementing Pre-built Informati...,"modelling method,introduction method,context-a...","[H, I, J]",model##method##introduct##method##inform##retr...,model##method##introduct##method##inform##retr...
6,8,A Methodology and a Toolkit that Integrate Tec...,"knowledge management,knowledge networks,inter-...","[C, I]",knowledg##manag##knowledg##network##network##b...,knowledg##manag##knowledg##network##network##b...
10,13,A Knowledge Infrastructure Hierarchy Model for...,"knowledge management,information systems,data ...",[H],knowledg##manag##inform##system##data##base,knowledg##manag##inform##system##data##base
12,15,Formal Construction of a Non-blocking Concurre...,"atomicity,concurrency,refinement,formal proof,...",[D],atom##concurr##refin##formal##proof##prover,atom##concurr##refin##formal##proof##prover
...,...,...,...,...,...,...
1105,1466,Effective Computability of Solutions of Differ...,"ordinary differential equations,differential i...","[F, G]",ordinari##differenti##equat##differenti##inclu...,ordinari##differenti##equat##differenti##inclu...
1106,1467,On the Effective Existence of Schauder Bases,"computatable functional analysis,Schauder basis","[F, G]",computat##function##analysi##schauder##basi,computat##function##analysi##schauder##basi
1109,1471,Application Scenarios for the Learning Objects...,"Learning Object,Learning Objects Repository,e-...","[H, K]",learn##object##learn##object##repositori##coll...,learn##object##learn##object##repositori##coll...
1111,1473,Eduquito: Virtual Environment for Digital Incl...,"virtual environment,accessibility,persons with...",[K],virtual##environ##access##person##disabl,virtual##environ##access##person##disabl


In [46]:
oldDf_FeaturesMap[0]

,Names,Metadata,system,comput,knowledg,model,inform,manag,languag,program,...,gather,gazett,gender,geodes,gif,recombin,recogn,glass,reattempt,zrtp
0,A,tf_keywords,5.0,1.0,27.0,2,6.0,22.0,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,5.0,1.0,27.0,2,6.0,22.0,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,10.0,2.0,54.0,4,12.0,44.0,12.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,1.0,8.0,0.0,2,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,1.0,8.0,0.0,2,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,2.0,16.0,0.0,4,0.0,0.0,6.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,16.0,13.0,3.0,12,4.0,4.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,C,tf_title,16.0,13.0,3.0,12,4.0,4.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,C,tf_keywords_title,32.0,26.0,6.0,24,8.0,8.0,6.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
9,D,tf_keywords,36.0,19.0,14.0,31,6.0,12.0,39.0,55.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
bestOffstring_list[0]

array([[ 4.93785354,  1.31357825, 35.25793261, ...,  0.        ,
         0.        ,  0.        ],
       [ 6.0973933 ,  1.21523604, 44.11729344, ...,  0.        ,
         0.        ,  0.        ],
       [ 5.56937405,  1.31051345, 35.36743861, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 5.61413792,  1.27458512, 38.10796073, ...,  0.        ,
         0.        ,  0.        ],
       [ 5.09566054,  1.31357825, 36.6691016 , ...,  0.        ,
         0.        ,  0.        ],
       [ 5.56937405,  1.31051345, 37.27100876, ...,  0.        ,
         0.        ,  0.        ]])

In [48]:
bestThresold_list

[0.1, 0.12, 0.1]

In [49]:
f_list

[[0.46974908263327103,
  0.4811194113223578,
  0.47045168224883926,
  0.4705641949074397,
  0.4733709575244788,
  0.47190146305375824,
  0.47327455453670664,
  0.4763090822708479,
  0.4780177381593255,
  0.47327455453670664],
 [0.4866042480840734,
  0.47908482909609656,
  0.48112145945878093,
  0.4831435947805434,
  0.482636043193469,
  0.4880892699799096,
  0.4880892699799096,
  0.4880892699799096,
  0.48517021494086904,
  0.4880892699799096],
 [0.4806091121714512,
  0.4797615251459367,
  0.4766689148965461,
  0.47812966534158063,
  0.4835915775052469,
  0.4807297898459755,
  0.4708086007630057,
  0.47838864402906134,
  0.4790434989491342,
  0.48020461601501563]]

In [50]:
# f_list = []
# for i in range(len(bestOffstring_list)):

#   dataFBRM_train_k, dataFBRM_train_t, dataFBRM_train_kt = getTFByDoc(train_list[i])
#   exp_list = train_list[i]['FirstLevel'].tolist()
#   datadf_chromosome = oldDf_FeaturesMap[i][oldDf_FeaturesMap[i]['Metadata'] != 'tf_keywords_title']
#   # print(datadf_chromosome)
#   df_features = datadf_chromosome.iloc[:, 2:]

#   f_ = getFittness(bestOffstring_list[i],oldDf_FeaturesMap[i],df_features.shape[1],nInd,bestOffstring_list[i].shape[0] * bestOffstring_list[i].shape[1],dataFBRM_train_kt,exp_list,bestThresold_list[i])
#   f_list.append(f_)


## TESTING

In [60]:
%%time
k = 3
f_test_list = []
p_test_bestPredictionList = []
e_test_exp_list = []

for i in range(len(bestOffstring_list)):

  dataFBRM_train_k, dataFBRM_train_t, dataFBRM_train_kt = getTFByDoc(test_list[i])
  exp_list = test_list[i]['FirstLevel'].tolist()

  ## TF
  dataFBRM_i = getTFByCategories(train_list[i])

  ## Create a Features Map
  df_featuresMap_i = getFeaturesMap(dataFBRM_i)

  ## hallar el df_featuresMap
  datadf_chromosome = df_featuresMap_i[df_featuresMap_i['Metadata'] != 'tf_keywords_title']

  df_features = datadf_chromosome.iloc[:, 2:]
  # display(df_features)

  # f_, predictionList = getFittness(bestOffstring_list[i],df_featuresMap_i,df_features.shape[1],nInd,bestOffstring_list[i].shape[1],dataFBRM_train_kt,exp_list,bestThresold_list[i])
  f_, predictionList = getFittness(bestOffstring_list[i],df_featuresMap_i,df_features.shape[1],nInd,bestOffstring_list[i].shape[1],dataFBRM_train_kt,exp_list,bestThresold_list[i])
  # f_, predictionList = getFittness(bestOffstring_list[i],df_featuresMap_i,df_features.shape[1],nInd,bestOffstring_list[i].shape[1],dataFBRM_train_kt,exp_list,0.1)

  f_test_list.append(f_)
  p_test_bestPredictionList.append(predictionList)
  e_test_exp_list.append(exp_list)



2


,Names,Metadata,system,comput,knowledg,model,inform,manag,languag,program,...,gather,gazett,gender,geodes,gif,recombin,recogn,glass,reattempt,zrtp
0,A,tf_keywords,5.0,1.0,27.0,2,6.0,22.0,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,5.0,1.0,27.0,2,6.0,22.0,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,10.0,2.0,54.0,4,12.0,44.0,12.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,1.0,8.0,0.0,2,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,1.0,8.0,0.0,2,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,2.0,16.0,0.0,4,0.0,0.0,6.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,16.0,13.0,3.0,12,4.0,4.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,C,tf_title,16.0,13.0,3.0,12,4.0,4.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,C,tf_keywords_title,32.0,26.0,6.0,24,8.0,8.0,6.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
9,D,tf_keywords,36.0,19.0,14.0,31,6.0,12.0,39.0,55.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



 ------------------------------------ POBLACION:  1  ------------------------------------ 


 ------------------------------------ POBLACION:  2  ------------------------------------ 


 ------------------------------------ POBLACION:  3  ------------------------------------ 


 ------------------------------------ POBLACION:  4  ------------------------------------ 


 ------------------------------------ POBLACION:  5  ------------------------------------ 


 ------------------------------------ POBLACION:  6  ------------------------------------ 


 ------------------------------------ POBLACION:  7  ------------------------------------ 


 ------------------------------------ POBLACION:  8  ------------------------------------ 


 ------------------------------------ POBLACION:  9  ------------------------------------ 


 ------------------------------------ POBLACION:  10  ------------------------------------ 

4


,Names,Metadata,system,knowledg,comput,inform,manag,model,languag,program,...,fit,flood,flowchart,focu,forens,riemannian,riemann,result,fubini,self
0,A,tf_keywords,1.0,22.0,0.0,10.0,13.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,1.0,22.0,0.0,10.0,13.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,2.0,44.0,0.0,20.0,26.0,0.0,14.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,2.0,0.0,6.0,0.0,0.0,2.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,2.0,0.0,6.0,0.0,0.0,2.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,4.0,0.0,12.0,0.0,0.0,4.0,8.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,24.0,11.0,18.0,9.0,15.0,7.0,4.0,5.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,C,tf_title,24.0,11.0,18.0,9.0,15.0,7.0,4.0,5.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,C,tf_keywords_title,48.0,22.0,36.0,18.0,30.0,14.0,8.0,10.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,D,tf_keywords,44.0,10.0,19.0,9.0,8.0,24.0,30.0,55.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0



 ------------------------------------ POBLACION:  1  ------------------------------------ 


 ------------------------------------ POBLACION:  2  ------------------------------------ 


 ------------------------------------ POBLACION:  3  ------------------------------------ 


 ------------------------------------ POBLACION:  4  ------------------------------------ 


 ------------------------------------ POBLACION:  5  ------------------------------------ 


 ------------------------------------ POBLACION:  6  ------------------------------------ 


 ------------------------------------ POBLACION:  7  ------------------------------------ 


 ------------------------------------ POBLACION:  8  ------------------------------------ 


 ------------------------------------ POBLACION:  9  ------------------------------------ 


 ------------------------------------ POBLACION:  10  ------------------------------------ 

2


,Names,Metadata,system,comput,knowledg,inform,manag,model,program,languag,...,hdr,head,hermitian,hertz,restor,heyt,hine,histor,hmm,semir
0,A,tf_keywords,6,1.0,33.0,12.0,21.0,2,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,6,1.0,33.0,12.0,21.0,2,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,12,2.0,66.0,24.0,42.0,4,2.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,1,8.0,0.0,0.0,0.0,2,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,1,8.0,0.0,0.0,0.0,2,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,2,16.0,0.0,0.0,0.0,4,4.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,20,13.0,8.0,7.0,11.0,7,4.0,5.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,C,tf_title,20,13.0,8.0,7.0,11.0,7,4.0,5.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,C,tf_keywords_title,40,26.0,16.0,14.0,22.0,14,8.0,10.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,D,tf_keywords,32,22.0,4.0,11.0,10.0,33,62.0,35.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0



 ------------------------------------ POBLACION:  1  ------------------------------------ 


 ------------------------------------ POBLACION:  2  ------------------------------------ 


 ------------------------------------ POBLACION:  3  ------------------------------------ 


 ------------------------------------ POBLACION:  4  ------------------------------------ 


 ------------------------------------ POBLACION:  5  ------------------------------------ 


 ------------------------------------ POBLACION:  6  ------------------------------------ 


 ------------------------------------ POBLACION:  7  ------------------------------------ 


 ------------------------------------ POBLACION:  8  ------------------------------------ 


 ------------------------------------ POBLACION:  9  ------------------------------------ 


 ------------------------------------ POBLACION:  10  ------------------------------------ 

CPU times: total: 48.8 s
Wall time: 52.7 s


In [61]:
f_list

[[0.46974908263327103,
  0.4811194113223578,
  0.47045168224883926,
  0.4705641949074397,
  0.4733709575244788,
  0.47190146305375824,
  0.47327455453670664,
  0.4763090822708479,
  0.4780177381593255,
  0.47327455453670664],
 [0.4866042480840734,
  0.47908482909609656,
  0.48112145945878093,
  0.4831435947805434,
  0.482636043193469,
  0.4880892699799096,
  0.4880892699799096,
  0.4880892699799096,
  0.48517021494086904,
  0.4880892699799096],
 [0.4806091121714512,
  0.4797615251459367,
  0.4766689148965461,
  0.47812966534158063,
  0.4835915775052469,
  0.4807297898459755,
  0.4708086007630057,
  0.47838864402906134,
  0.4790434989491342,
  0.48020461601501563]]

In [62]:
f_test_list

[[0.34645023503617733,
  0.3521100252369283,
  0.3495776975280519,
  0.3508897980987466,
  0.3458278226102197,
  0.35032140103928,
  0.34597663410855656,
  0.35082080608087407,
  0.3541661706351921,
  0.34597663410855656],
 [0.3212133802434546,
  0.3218666051954753,
  0.3178791850647491,
  0.3147687824283426,
  0.3117523123545105,
  0.322140867194581,
  0.322140867194581,
  0.322140867194581,
  0.32375687900858313,
  0.322140867194581],
 [0.351232609856375,
  0.3584617856414273,
  0.34632640791743646,
  0.342871485209882,
  0.34508169183832604,
  0.35432052682696175,
  0.33983714657773895,
  0.33726707391120675,
  0.34038627802692045,
  0.3406612216273215]]

# Busqueda del nuevo threshold en los offtrings

In [63]:
idxBestResultTrain = []
bestThresoldNew = []
bestOffstringIdx = []

for i in range(len(f_list)):
  maxV = 0
  idxMax = 0
  for j in range(len(f_list[i])):
    if(f_list[i][j] > maxV):
      maxV = f_list[i][j]
      idxMax = j

  ## TF by Doc
  dataFBRM_train_k, dataFBRM_train_t, dataFBRM_train_kt = getTFByDoc(train_list[i])
  exp_list = train_list[i]['FirstLevel'].tolist()

  ## TF
  dataFBRM_i = getTFByCategories(train_list[i])

  ## Create a Features Map
  df_featuresMap_i = getFeaturesMap(dataFBRM_i)

  ## hallar el df_featuresMap
  datadf_chromosome = df_featuresMap_i[df_featuresMap_i['Metadata'] != 'tf_keywords_title']

  df_features = datadf_chromosome.iloc[:, 2:]
  # display(df_features)


  candidateGeneration = [bestOffstring_list[i][idxMax]]
  fitnessByChromosome, thresholdByChromosome = getThresold(candidateGeneration,df_featuresMap,df_features.shape[1],nInd,len(candidateGeneration),dataFBRM_train_kt,exp_list,threshold)
  print("mejores resultados: ",fitnessByChromosome, "\t-> mejor thresold",thresholdByChromosome)
  idx_max= fitnessByChromosome.index(max(fitnessByChromosome))
  bestThresold = thresholdByChromosome[idx_max]
  bestThresoldNew.append(bestThresold)
  bestOffstringIdx.append(idxMax)

2


,Names,Metadata,system,comput,knowledg,model,inform,manag,languag,program,...,gather,gazett,gender,geodes,gif,recombin,recogn,glass,reattempt,zrtp
0,A,tf_keywords,5.0,1.0,27.0,2,6.0,22.0,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,5.0,1.0,27.0,2,6.0,22.0,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,10.0,2.0,54.0,4,12.0,44.0,12.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,1.0,8.0,0.0,2,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,1.0,8.0,0.0,2,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,2.0,16.0,0.0,4,0.0,0.0,6.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,16.0,13.0,3.0,12,4.0,4.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,C,tf_title,16.0,13.0,3.0,12,4.0,4.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,C,tf_keywords_title,32.0,26.0,6.0,24,8.0,8.0,6.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
9,D,tf_keywords,36.0,19.0,14.0,31,6.0,12.0,39.0,55.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



 ------------------------------------ POBLACION:  1  ------------------------------------ 

Con threshold:  0.1
the best Thresold:  0.1
the best Acurracy:  0.17356676149633873
Con threshold:  0.11
Con threshold:  0.12
Con threshold:  0.13
Con threshold:  0.14
Con threshold:  0.15000000000000002
Con threshold:  0.16000000000000003
mejores resultados:  [0.17356676149633873] 	-> mejor thresold [0.1]
4


,Names,Metadata,system,knowledg,comput,inform,manag,model,languag,program,...,fit,flood,flowchart,focu,forens,riemannian,riemann,result,fubini,self
0,A,tf_keywords,1.0,22.0,0.0,10.0,13.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,1.0,22.0,0.0,10.0,13.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,2.0,44.0,0.0,20.0,26.0,0.0,14.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,2.0,0.0,6.0,0.0,0.0,2.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,2.0,0.0,6.0,0.0,0.0,2.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,4.0,0.0,12.0,0.0,0.0,4.0,8.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,24.0,11.0,18.0,9.0,15.0,7.0,4.0,5.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,C,tf_title,24.0,11.0,18.0,9.0,15.0,7.0,4.0,5.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,C,tf_keywords_title,48.0,22.0,36.0,18.0,30.0,14.0,8.0,10.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,D,tf_keywords,44.0,10.0,19.0,9.0,8.0,24.0,30.0,55.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0



 ------------------------------------ POBLACION:  1  ------------------------------------ 

Con threshold:  0.1
the best Thresold:  0.1
the best Acurracy:  0.18794073011443885
Con threshold:  0.11
Con threshold:  0.12
Con threshold:  0.13
Con threshold:  0.14
Con threshold:  0.15000000000000002
Con threshold:  0.16000000000000003
mejores resultados:  [0.18794073011443885] 	-> mejor thresold [0.1]
2


,Names,Metadata,system,comput,knowledg,inform,manag,model,program,languag,...,hdr,head,hermitian,hertz,restor,heyt,hine,histor,hmm,semir
0,A,tf_keywords,6,1.0,33.0,12.0,21.0,2,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,6,1.0,33.0,12.0,21.0,2,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,12,2.0,66.0,24.0,42.0,4,2.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,1,8.0,0.0,0.0,0.0,2,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,1,8.0,0.0,0.0,0.0,2,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,2,16.0,0.0,0.0,0.0,4,4.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,20,13.0,8.0,7.0,11.0,7,4.0,5.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,C,tf_title,20,13.0,8.0,7.0,11.0,7,4.0,5.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,C,tf_keywords_title,40,26.0,16.0,14.0,22.0,14,8.0,10.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,D,tf_keywords,32,22.0,4.0,11.0,10.0,33,62.0,35.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0



 ------------------------------------ POBLACION:  1  ------------------------------------ 

Con threshold:  0.1
the best Thresold:  0.1
the best Acurracy:  0.4835915775052469
Con threshold:  0.11
Con threshold:  0.12
Con threshold:  0.13
Con threshold:  0.14
Con threshold:  0.15000000000000002
Con threshold:  0.16000000000000003
mejores resultados:  [0.4835915775052469] 	-> mejor thresold [0.1]


# Seleecionando los mejores resultados

In [64]:
idxBestResultTest = []

for i in range(len(f_test_list)):
  maxV = 0
  idxMax = 0
  for j in range(len(f_test_list[i])):
    if (f_test_list[i][j] > maxV):
      maxV = f_test_list[i][j]
      idxMax = j
  idxBestResultTest.append(idxMax)


In [65]:
idxBestResultTest

[8, 8, 1]

# ACCURACY

In [66]:
medianAcurracy = 0
for i in range(len(idxBestResultTest)):
  medianAcurracy += f_test_list[i][idxBestResultTest[i]]
print(medianAcurracy/len(idxBestResultTest))

0.3454616117617342


In [67]:
print(len(p_test_bestPredictionList[0]))
print(len(e_test_exp_list[0]))


10
372


In [68]:
for i in range(len(idxBestResultTest)):
  predList = p_test_bestPredictionList[i][idxBestResultTest[i]]
  expList = e_test_exp_list[i]
  # print(i)
  # print(len(expList))
  print("THE BEST RESULT")
  for j in range(len(predList)):
    print("y_exp: ",expList[j] , "\t->", predList[j])


THE BEST RESULT
y_exp:  ['A' 'D' 'H' 'J' 'K'] 	-> ['H', 'I', 'K']
y_exp:  ['H' 'I' 'J'] 	-> ['D', 'H', 'I', 'K']
y_exp:  ['C' 'I'] 	-> ['H', 'I', 'K']
y_exp:  ['H'] 	-> ['D', 'H', 'I', 'K']
y_exp:  ['D'] 	-> ['D', 'F']
y_exp:  ['H'] 	-> ['D', 'H', 'I']
y_exp:  ['F' 'I'] 	-> ['D', 'F', 'H', 'I']
y_exp:  ['K'] 	-> ['C', 'D', 'H', 'K']
y_exp:  ['D' 'H'] 	-> ['D', 'E', 'H', 'I']
y_exp:  ['C' 'D' 'K'] 	-> ['C', 'D', 'F', 'H', 'K']
y_exp:  ['H'] 	-> ['D', 'F', 'H']
y_exp:  ['H' 'I' 'K'] 	-> ['H', 'I', 'K']
y_exp:  ['H'] 	-> ['D', 'H', 'I', 'K']
y_exp:  ['H'] 	-> ['D', 'F', 'H']
y_exp:  ['C'] 	-> ['A', 'D', 'H', 'K']
y_exp:  ['B' 'F'] 	-> ['C', 'D', 'F', 'H', 'I']
y_exp:  ['D'] 	-> ['D', 'F']
y_exp:  ['E'] 	-> ['D', 'F', 'H', 'I', 'K']
y_exp:  ['H'] 	-> ['H', 'I', 'K']
y_exp:  ['D'] 	-> ['D', 'F', 'H']
y_exp:  ['E'] 	-> ['F', 'H']
y_exp:  ['I'] 	-> ['D', 'H', 'I', 'K']
y_exp:  ['B'] 	-> ['D', 'F', 'H', 'I']
y_exp:  ['D'] 	-> ['D', 'H']
y_exp:  ['D' 'F'] 	-> ['D', 'F', 'H', 'I', 'K']
y_exp:  [

# Extrayendo el Mapa de TF

In [71]:

display(oldDf_FeaturesMap[0])
df_newFM = oldDf_FeaturesMap[0].copy()

population = bestOffstring_list[0][idxBestResultTest[0]]

# ## hallar el df_featuresMap
# datadf_chromosome = df_newFM[df_newFM['Metadata'] != 'tf_keywords_title']

df_features = df_newFM.iloc[:, 2:]
df_categories_metadata = df_newFM.iloc[:, :2]
# print(df_features.shape[0])

display(df_features)
display(df_categories_metadata)


# columns = list(df_featuresMap.iloc[:, 2:].columns)
nRow = len(population) // df_features.shape[1]

blocks = np.split(population, nRow)
print(blocks[0])

i = 0
k = 0
while(i < 39):
  for j in range(len(blocks[k])):
    df_features.iloc[i, j] = blocks[k][j]
  k+=2
  i+=3

i = 1
k = 1
while(i < 39):
  for j in range(len(blocks[k])):
    df_features.iloc[i, j] = blocks[k][j]
  k+=2
  i+=3

i = 2
k = 0
while(i < 39):
  for j in range(len(blocks[k])):
    df_features.iloc[i, j] = blocks[k][j] + blocks[k+1][j]
  k+=2
  i+=3



,Names,Metadata,system,comput,knowledg,model,inform,manag,languag,program,...,gather,gazett,gender,geodes,gif,recombin,recogn,glass,reattempt,zrtp
0,A,tf_keywords,5.0,1.0,27.0,2,6.0,22.0,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,5.0,1.0,27.0,2,6.0,22.0,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,10.0,2.0,54.0,4,12.0,44.0,12.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,1.0,8.0,0.0,2,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,1.0,8.0,0.0,2,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,2.0,16.0,0.0,4,0.0,0.0,6.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,16.0,13.0,3.0,12,4.0,4.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,C,tf_title,16.0,13.0,3.0,12,4.0,4.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,C,tf_keywords_title,32.0,26.0,6.0,24,8.0,8.0,6.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
9,D,tf_keywords,36.0,19.0,14.0,31,6.0,12.0,39.0,55.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,system,comput,knowledg,model,inform,manag,languag,program,formal,learn,...,gather,gazett,gender,geodes,gif,recombin,recogn,glass,reattempt,zrtp
0,5.0,1.0,27.0,2,6.0,22.0,6.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.0,1.0,27.0,2,6.0,22.0,6.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.0,2.0,54.0,4,12.0,44.0,12.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,8.0,0.0,2,0.0,0.0,3.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,8.0,0.0,2,0.0,0.0,3.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.0,16.0,0.0,4,0.0,0.0,6.0,2.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,16.0,13.0,3.0,12,4.0,4.0,3.0,3.0,10.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,16.0,13.0,3.0,12,4.0,4.0,3.0,3.0,10.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,32.0,26.0,6.0,24,8.0,8.0,6.0,6.0,20.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
9,36.0,19.0,14.0,31,6.0,12.0,39.0,55.0,36.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Names,Metadata
0,A,tf_keywords
1,A,tf_title
2,A,tf_keywords_title
3,B,tf_keywords
4,B,tf_title
5,B,tf_keywords_title
6,C,tf_keywords
7,C,tf_title
8,C,tf_keywords_title
9,D,tf_keywords


[ 5.09566054  1.31357825 36.6691016  ...  0.          0.
  0.        ]


In [72]:
df_OffString = pd.concat([df_categories_metadata, df_features], axis=1)
df_OffString

,Names,Metadata,system,comput,knowledg,model,inform,manag,languag,program,...,gather,gazett,gender,geodes,gif,recombin,recogn,glass,reattempt,zrtp
0,A,tf_keywords,5.095661,1.313578,36.669102,3.565371,8.574643,29.401742,9.837586,1.461380,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,A,tf_title,7.635377,1.138116,26.799691,3.084502,6.092118,23.180565,9.058034,1.141141,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,A,tf_keywords_title,12.731037,2.451694,63.468793,6.649873,14.666762,52.582308,18.895620,2.602521,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,B,tf_keywords,1.266036,17.394837,0.000000,1.920604,0.000000,0.000000,3.263912,1.054907,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,B,tf_title,1.344782,9.192273,0.000000,2.506442,0.000000,0.000000,4.438473,1.682491,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,B,tf_keywords_title,2.610818,26.587110,0.000000,4.427046,0.000000,0.000000,7.702385,2.737398,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,C,tf_keywords,11.594478,18.075845,3.921724,18.569056,4.165525,4.761822,3.422832,3.018543,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.858412,0.000000
7,C,tf_title,20.043157,17.079267,3.272954,22.409143,5.707363,5.080903,5.431859,4.339438,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.386200,0.000000
8,C,tf_keywords_title,31.637635,35.155112,7.194678,40.978199,9.872888,9.842725,8.854691,7.357982,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.244612,0.000000
9,D,tf_keywords,35.099570,24.349477,25.070089,29.818893,6.064103,13.648170,61.604432,82.118183,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
